# Import Libraries and Dataset

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 



In [ ]:
# Load the TSV file
file_path = 'C:/Users/ANKITA/Downloads/amazon_alexa.tsv'
reviews_df = pd.read_csv(file_path, sep='\t')



In [ ]:
reviews_df



In [ ]:
reviews_df.info()

In [ ]:
reviews_df.describe()

In [ ]:
reviews_df['verified_reviews']

# Exploring Dataset

In [ ]:
 sns.heatmap(reviews_df.isnull(), yticklabels = False, cbar = False, cmap="Blues")

In [ ]:
reviews_df.hist(bins = 30, figsize = (13,5), color = 'r')

In [ ]:
reviews_df['verified_reviews'] = reviews_df['verified_reviews'].fillna('').astype(str)


In [ ]:
reviews_df['length'] = reviews_df['verified_reviews'].apply(len)

In [ ]:
reviews_df.head()

In [ ]:
reviews_df['length'].plot(bins=100, kind='hist') 

In [ ]:
reviews_df.length.describe()

In [ ]:
# Let's see the longest message 43952
reviews_df[reviews_df['length'] == 2851]['verified_reviews'].iloc[0]

In [ ]:
# Let's see the shortest message 
reviews_df[reviews_df['length'] == 1]['verified_reviews'].iloc[0]

In [ ]:
# Let's see the message with mean length 
reviews_df[reviews_df['length'] == 133]['verified_reviews'].iloc[0]

In [ ]:
positive = reviews_df[reviews_df['feedback']==1]
negative = reviews_df[reviews_df['feedback']==0]

In [ ]:
positive

In [ ]:
negative

In [ ]:
 sns.countplot(x='feedback', data=reviews_df)

In [ ]:
sns.countplot(x = 'rating', data = reviews_df)

In [ ]:
reviews_df['rating'].hist(bins = 5)

In [ ]:
plt.figure(figsize = (40,15))
sns.barplot(x = 'variation', y='rating', data = reviews_df, palette = 'deep')

In [ ]:
sentences = reviews_df['verified_reviews'].tolist()
len(sentences)

In [ ]:
print(sentences)

In [ ]:
sentences_as_one_string =" ".join(sentences)

In [ ]:
sentences_as_one_string

In [ ]:
from wordcloud import WordCloud

plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(sentences_as_one_string))

In [ ]:
negative_list = negative['verified_reviews'].tolist()

negative_list

In [ ]:
negative_sentences_as_one_string = " ".join(negative_list)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(negative_sentences_as_one_string))

# PERFORM DATA CLEANING

In [ ]:
# Let's drop the date, rating, length
reviews_df = reviews_df.drop(['date', 'rating', 'length'],axis=1)


In [ ]:
reviews_df

In [ ]:
variation_dummies = pd.get_dummies(reviews_df['variation'], drop_first = True)
# Avoid Dummy Variable trap which occurs when one variable can be predicted from the other.

In [ ]:
variation_dummies

In [ ]:
# first let's drop the column
reviews_df.drop(['variation'], axis=1, inplace=True)

In [ ]:
# Now let's add the encoded column again
reviews_df = pd.concat([reviews_df, variation_dummies], axis=1)

In [ ]:
reviews_df

# REMOVING PUNCTUATION FROM TEXT

In [ ]:
import string
string.punctuation

import nltk
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')


In [ ]:
# Let's define a pipeline to clean up all the messages 
# The pipeline performs the following: (1) remove punctuation, (2) remove stopwords

def message_cleaning(message):
    Test_punc_removed = [char for char in message if char not in string.punctuation]
    Test_punc_removed_join = ''.join(Test_punc_removed)
    Test_punc_removed_join_clean = [word for word in Test_punc_removed_join.split() if word.lower() not in stopwords.words('english')]
    return Test_punc_removed_join_clean

In [ ]:
# Let's test the newly added function
reviews_df_clean = reviews_df['verified_reviews'].apply(message_cleaning)

In [ ]:
print(reviews_df_clean[3]) # show the cleaned up version

In [ ]:
print(reviews_df['verified_reviews'][3]) # show the original version

In [ ]:
reviews_df_clean

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Define the cleaning pipeline we defined earlier
vectorizer = CountVectorizer(analyzer = message_cleaning)
reviews_countvectorizer = vectorizer.fit_transform(reviews_df['verified_reviews'])

In [ ]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names)

In [ ]:
print(reviews_countvectorizer.toarray())  

In [ ]:
reviews_countvectorizer.shape

In [ ]:
reviews_df

In [ ]:
# first let's drop the column
reviews_df.drop(['verified_reviews'], axis=1, inplace=True)
reviews = pd.DataFrame(reviews_countvectorizer.toarray())

In [ ]:
# Now let's concatenate them together
reviews_df = pd.concat([reviews_df, reviews], axis=1)

In [ ]:
reviews_df

In [ ]:
# Let's drop the target label coloumns
X = reviews_df.drop(['feedback'],axis=1)

In [ ]:
X

In [ ]:
y = reviews_df['feedback']

#  Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# Naive Bayes Training and Evaluation


In [ ]:
# Trains a Naive Bayes model.

NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)

y_predict_train = NB_classifier.predict(X_train)


In [ ]:
cm = confusion_matrix(y_train, y_predict_train)
sns.heatmap(cm, annot=True)


In [ ]:
# Displays a confusion matrix for training predictions.

y_predict_test = NB_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_predict_test)
sns.heatmap(cm, annot=True)
print(classification_report(y_test, y_predict_test))


# Logistic Regression Training and Evaluation



In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)


In [ ]:
# Testing Set Performance
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print('Accuracy {} %'.format( 100 * accuracy_score(y_pred, y_test)))


In [ ]:
cm = confusion_matrix(y_pred, y_test)
sns.heatmap(cm, annot = True)

In [ ]:
print(classification_report(y_test, y_pred))


## Final Conclusion

This project focused on analyzing Amazon Alexa customer reviews to predict whether customers are satisfied based on their written feedback.

### Key Objectives:
- Determine if a customer is **satisfied (1)** or **not satisfied (0)** using the review text.
- Apply Natural Language Processing (NLP) techniques and machine learning models to build a reliable sentiment classifier.

### What We Did:
- Cleaned and preprocessed the review data by removing punctuation and stopwords
- Explored the dataset through visualizations (word clouds, rating distribution, feedback count)
- Converted text data into numeric features using `CountVectorizer`
- Used one-hot encoding for product variation
- Built and evaluated two models:
  - **Naive Bayes**
  - **Logistic Regression**

### Results:
- **Logistic Regression** performed best with an accuracy of **~95%**
- Most customers in the dataset were **satisfied**, resulting in some class imbalance
- Models struggled slightly with predicting the minority class (unsatisfied reviews)

### Final Insight:
**Yes, the majority of customers were satisfied with Amazon Alexa**, as shown by both the data distribution and model predictions.

This project demonstrates how textual reviews can be effectively used to gain insights into customer sentiment using NLP and machine learning. 

